In [338]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [339]:
from rushhour_objects import *

In [340]:
from copy import deepcopy

## Simple examples using 'Car'

In [341]:
# Car placed horizontally
car = Car(color="green",position=[0,0],length=3,orientation="h")
print("\nOriginal       : %s"%car)

# Displacement along the 'free' axis
car += 1
print("Displacement +1: %s"%car)
car -= 1
print("Displacement -1: %s"%car)

# Car placed vertically
car = Car(color="green",position=[0,0],length=3,orientation="v")
print("\nOriginal       : %s"%car)

# Displacement along the 'free' axis
car += 1
print("Displacement +1: %s"%car)
car -= 1
print("Displacement -1: %s"%car)


# Assume that a car is placed in a board of dimension 6x6
print("\nChecking the acceptance of moves")
print("Original       : %s"%car)
for i in range(6):
    if car.can_move(+1,6): 
        car += 1
        print("- Move accepted: %s"%car)
    else:
        print("- Move denied  : %s"%car)
        
# Moving back again (lower limit is 0)
for i in range(6):
    if car.can_move(-1,0): 
        car -= 1
        print("- Move accepted: %s"%car)
    else:
        print("- Move denied  : %s"%car)


Original       : Instance of Car(color=green,position=(0, 0),length=3,orientation=h)
Displacement +1: Instance of Car(color=green,position=(0, 1),length=3,orientation=h)
Displacement -1: Instance of Car(color=green,position=(0, 0),length=3,orientation=h)

Original       : Instance of Car(color=green,position=(0, 0),length=3,orientation=v)
Displacement +1: Instance of Car(color=green,position=(1, 0),length=3,orientation=v)
Displacement -1: Instance of Car(color=green,position=(0, 0),length=3,orientation=v)

Checking the acceptance of moves
Original       : Instance of Car(color=green,position=(0, 0),length=3,orientation=v)
- Move accepted: Instance of Car(color=green,position=(1, 0),length=3,orientation=v)
- Move accepted: Instance of Car(color=green,position=(2, 0),length=3,orientation=v)
- Move accepted: Instance of Car(color=green,position=(3, 0),length=3,orientation=v)
- Move denied  : Instance of Car(color=green,position=(3, 0),length=3,orientation=v)
- Move denied  : Instance of 

### Dealing with the position of the car in terms of slices

In [344]:
# Create a board
bb = np.zeros([6,6])
print("Original\n%s"%bb)

# Three cars in different positions 
car1 = Car(color="green",position=[0,0],length=3,orientation="h")
car2 = Car(color="green",position=[1,1],length=3,orientation="h")
car3 = Car(color="green",position=[3,5],length=3,orientation="v")
bb[car1.npindices] = 1
bb[car2.npindices] = 1
bb[car3.npindices] = 1
print("Placement\n%s"%bb)



Original
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Placement
[[1. 1. 1. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]


## Board manipulations

In [323]:
# Create the board
board = Board(5,2)

# Insert some cars (more cross validation needed probably) 
board.insert_car("green",[0,0],3,"v")
board.insert_car("red",[0,1],3,"v")
board.insert_car("magenta",[3,2],3,"h")

# Always a good practice to check the following line 
occupied_positions = 3 + 3 + 3
if occupied_positions != board.empty_places:
    raise ValueError("Something went wrong in insersion")
board

Board with view
[[1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 1 1 1]
 [0 0 0 0 0]]
containing 3 cars

### Current state (and check if it is hashable)

In [327]:
state = board.get_state()
try:
    print("Hash of state: %s"%hash(state))
except TypeError:
    print("State is not hashable")

Hash of state: 417673259425745701


### Build a new board from a given state

In [329]:
board2 = Board.from_state(state)
board2

Board with view
[[1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 1 1 1]
 [0 0 0 0 0]]
containing 3 cars

### Get all connected states with the currect Board

In [335]:
for item in board.connected_states:
    # Since item is a state I will create new boards
    print(Board.from_state(item),"\n")

Board with view
[[0 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 0 1 1 1]
 [0 0 0 0 0]]
containing 3 cars 

Board with view
[[1 0 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [0 1 1 1 1]
 [0 0 0 0 0]]
containing 3 cars 

Board with view
[[1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [0 1 1 1 0]
 [0 0 0 0 0]]
containing 3 cars 



In [336]:
# But I do not change the original board at all!!
board

Board with view
[[1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 1 1 1]
 [0 0 0 0 0]]
containing 3 cars

### Explicit displacement and update of the view (avoid)

In [337]:
board["green"] = 1   # --> means +1 displacement in the green car
board

Board with view
[[0 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 0 1 1 1]
 [0 0 0 0 0]]
containing 3 cars